In [24]:
import numpy as np
import ipywidgets as iw
from brepmatching.data import BRepMatchingDataset, load_data
import torch
import meshplot
from IPython.display import clear_output, display, HTML
from functools import partial

In [2]:
data_path = '/fast/jamesn8/brepmatching/TopoV2FullRun.zip'
data_cache = '/fast/jamesn8/brepmatching/TopoV2FullRun.pt'

cache_data = load_data(data_path, data_cache)
data = BRepMatchingDataset(cache_data, mode='test') # TODO: make sure it's using the correct set/groups

In [22]:
out = iw.Output(layout={'border': '1px solid black'})

@out.capture(clear_output=True, wait=True)
def process_sample(data_ind):
    brep = data[data_ind]
    indices = {}
    for topo_type in ['faces', 'edges', 'vertices']:
        num_right_topos = getattr(brep, 'right_' + topo_type).shape[0]
        exact_matches = getattr(brep, 'bl_exact_' + topo_type + '_matches')
        right_matchless_mask = torch.ones(num_right_topos, dtype=torch.int64)
        right_matchless_mask[exact_matches[1]] = 0
        #TODO: Also mask out the ones for which predictions agree (once you have those)
        #TODO: Baseline heuristic (how to obtain this?)
        indices[topo_type] = right_matchless_mask.nonzero()

    shading = {"flat":True, # Flat or smooth shading of triangles
           #"wireframe":True, "wire_width": 0.03, "wire_color": "black", # Wireframe rendering
           "width": 400, "height": 400, # Size of the viewer canvas
           "antialias": True, # Antialising, might not work on all GPUs
           "scale": 2.0, # Scaling of the model
           "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
           "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
           "background": "#ffffff", # Background color of the canvas
           "line_width": 1.0, "line_color": "black", # Line properties of overlay lines
           "bbox": False, # Enable plotting of bounding box
           "point_color": "red", "point_size": 0.01 # Point properties of overlay points
          }
    plot_left = meshplot.plot(brep.left_V.numpy(), brep.left_F.T.numpy(), shading=shading, return_plot=True)
    plot_right = meshplot.plot(brep.right_V.numpy(), brep.right_F.T.numpy(), shading=shading, return_plot=True)
    #display(iw.HBox([plot_left, plot_right]))
    display(plot_left)
    display(plot_right)
    #html_left = plot_left.to_html(imports=True, html_frame=False)
    #html_right = plot_right.to_html(imports=True, html_frame=False)
    #display(HTML(html_left))
    #display(HTML(html_right))



In [27]:
curr_ind = 0

button_next = iw.Button(description='Next')
button_prev = iw.Button(description='Previous')
def on_button_clicked(b, advance=True):
    global curr_ind
    #global button

    #out.clear_output()
    #display(button)
    if advance:
        curr_ind += 1 #TODO: Start correctly
    else:
        curr_ind -= 1
    print(curr_ind)
    process_sample(curr_ind)

button_next.on_click(partial(on_button_clicked, advance=True))
button_prev.on_click(partial(on_button_clicked, advance=False))
display(iw.HBox([button_prev, button_next]))
display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…